# Meta 패밀리 모델로 빌드하기

## 소개

이 강의에서는 다음 내용을 다룹니다:

- Meta 패밀리의 두 가지 주요 모델인 Llama 3.1과 Llama 3.2 살펴보기
- 각 모델의 사용 사례와 적용 시나리오 이해하기
- 각 모델의 고유한 기능을 보여주는 코드 예제

## Meta 패밀리 모델

이번 강의에서는 Meta 패밀리, 즉 "Llama Herd"의 두 가지 모델인 Llama 3.1과 Llama 3.2를 살펴봅니다.

이 모델들은 다양한 버전으로 제공되며, Github Model 마켓플레이스에서 사용할 수 있습니다. Github Models를 활용해 [AI 모델로 프로토타입 만들기](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)에 대한 자세한 내용도 참고하세요.

모델 버전:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*참고: Llama 3도 Github Models에서 제공되지만, 이번 강의에서는 다루지 않습니다*


## Llama 3.1

4050억 개의 파라미터를 가진 Llama 3.1은 오픈 소스 LLM 범주에 속합니다.

이 모델은 이전에 출시된 Llama 3에서 다음과 같은 업그레이드를 제공합니다:

- 더 커진 컨텍스트 윈도우 - 128k 토큰(기존 8k 토큰 대비)
- 더 많은 최대 출력 토큰 - 4096(기존 2048 대비)
- 향상된 다국어 지원 - 학습 토큰 수 증가로 인한 효과

이러한 기능 덕분에 Llama 3.1은 GenAI 애플리케이션을 구축할 때 더 복잡한 사용 사례를 처리할 수 있습니다. 예를 들어:
- 네이티브 함수 호출 - LLM 워크플로우 외부의 외부 도구 및 함수 호출 가능
- 향상된 RAG 성능 - 더 넓어진 컨텍스트 윈도우 덕분
- 합성 데이터 생성 - 파인튜닝과 같은 작업에 효과적인 데이터를 생성할 수 있는 능력


### 네이티브 함수 호출

Llama 3.1은 함수나 도구 호출을 더 효과적으로 할 수 있도록 추가로 튜닝되었습니다. 또한, 모델이 사용자 프롬프트를 바탕으로 필요하다고 판단할 때 사용할 수 있는 두 가지 내장 도구가 있습니다. 이 도구들은 다음과 같습니다.

- **Brave Search** - 웹 검색을 통해 날씨와 같은 최신 정보를 얻을 때 사용할 수 있습니다.
- **Wolfram Alpha** - 복잡한 수학 계산을 할 때 사용할 수 있어, 직접 함수를 작성할 필요가 없습니다.

또한, LLM이 호출할 수 있는 나만의 커스텀 도구를 만들 수도 있습니다.

아래 코드 예시에서는:

- 시스템 프롬프트에서 사용 가능한 도구(brave_search, wolfram_alpha)를 정의합니다.
- 특정 도시의 날씨를 묻는 사용자 프롬프트를 보냅니다.
- LLM은 Brave Search 도구를 호출하는 형태로 응답하며, 이는 다음과 같이 보입니다: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*참고: 이 예시는 도구 호출만 수행합니다. 결과를 얻으려면 Brave API 페이지에서 무료 계정을 만들고, 직접 함수를 정의해야 합니다.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Llama 3.1은 LLM임에도 불구하고 멀티모달리티라는 한계가 있습니다. 즉, 이미지를 비롯한 다양한 입력을 프롬프트로 사용하고 이에 대한 응답을 제공하는 기능이 부족합니다. 이 기능이 바로 Llama 3.2의 주요 특징 중 하나입니다. 주요 기능은 다음과 같습니다:

- 멀티모달리티 - 텍스트와 이미지 프롬프트 모두를 평가할 수 있는 기능
- 소형 및 중형 모델(11B와 90B) - 유연한 배포 옵션 제공
- 텍스트 전용 모델(1B와 3B) - 엣지/모바일 기기에서 모델을 배포할 수 있고, 지연 시간이 낮음

멀티모달 지원은 오픈소스 모델 분야에서 큰 진전입니다. 아래 코드 예시는 이미지와 텍스트 프롬프트를 함께 사용하여 Llama 3.2 90B로부터 이미지 분석을 받는 방법을 보여줍니다.

### Llama 3.2의 멀티모달 지원


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## 학습은 여기서 끝나지 않습니다, 여정을 계속하세요

이 강의를 마친 후에는 [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst)을 확인하여 생성형 AI에 대한 지식을 계속해서 쌓아보세요!



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서(원어)가 공식적인 기준임을 유의하시기 바랍니다. 중요한 정보의 경우, 전문 인간 번역을 권장합니다. 본 번역 사용으로 인해 발생하는 오해나 오역에 대해 당사는 책임을 지지 않습니다.
